In [ ]:
# First set of functions recursively mine dataset in folder CT_IMAGE.nrrd structures > mask_name_0...mask_name_n
# convert to nnunetv2 dataset structure...
# for most of our cases we will be using the simple dataset structure defined in
# https://github.com/MIC-DKFZ/nnUNet/blob/master/documentation/dataset_format.md
# Here is another example of the second dataset of the MSD, which has only one input channel:
# nnUNet_raw/Dataset002_Heart/
# ├── dataset.json
# ├── imagesTr
# │   ├── la_003_0000.nii.gz
# │   ├── la_004_0000.nii.gz
# │   ├── ...
# ├── imagesTs
# │   ├── la_001_0000.nii.gz
# │   ├── la_002_0000.nii.gz
# │   ├── ...
# └── labelsTr
#     ├── la_003.nii.gz
#     ├── la_004.nii.gz
#     ├── ...


In [2]:
import pandas as pd
import glob, os, numpy as np, nrrd

dataset_name = "Dataset001_HNOAR"
try:
    os.mkdir("/Users/jmarsill/Documents/nnunetV2")
    os.mkdir(f"/Users/jmarsill/Documents/nnunetV2/{dataset_name}")
    os.mkdir(f"/Users/jmarsill/Documents/nnunetV2/{dataset_name}/imagesTr")
    os.mkdir(f"/Users/jmarsill/Documents/nnunetV2/{dataset_name}/imagesTs")
    os.mkdir(f"/Users/jmarsill/Documents/nnunetV2/{dataset_name}/labelsTr")
    os.mkdir(f"/Users/jmarsill/Documents/nnunetV2/{dataset_name}/labelsTs")
except:
    pass

paths = ["/Users/jmarsill/Documents/External Data/WOLENS/Structured/DEEPMIND:PDDCA",
         "/Users/jmarsill/Documents/External Data/WOLENS/Structured/StructSeg2019",
         "/Users/jmarsill/Documents/External Data/WOLENS/Structured/TCIA-HNSCC-3DCT"]

count = 1
for p in paths:
    folders = glob.glob(p+"/*")
    for f in folders:
        patient = f.split("/")[-1]
        rois = glob.glob(f + "/structures/*")
        ending = rois[0].split(".")[-1]
        rois = [r.split("/")[-1].partition(".")[0] for r in rois]
        rois.sort()
        break
        # read the image...its the same...
        img = nrrd.read(f+"/CT_IMAGE.nrrd")
        head = img[1]
        
        if len(str(count)) < 3:
            val_count = "0" + str(count)
            if len(val_count) < 3:
                val_count = "0" + val_count
        else:
            val_count = str(count)
                
        new_img_path = f"/Users/jmarsill/Documents/nnunetV2/{dataset_name}/imagesTr/{patient}_{val_count}_000.nrrd"
        nrrd.write(new_img_path, img[0], img[1])
        
        mask = None
        for i, roi in enumerate(rois):
            if mask is None:
                mask = nrrd.read(f + f"/structures/{roi}.{ending}")
                mask = mask[0]
            else:
                mask_ = nrrd.read(f + f"/structures/{roi}.{ending}")
                mask_ = mask_[0]
                mask[mask_==1] = i+1
        
        print(f"Finished making {mask.max()} class mask for {patient}")
        new_mask_path = f"/Users/jmarsill/Documents/nnunetV2/{dataset_name}/labelsTr/{patient}_{val_count}_000.nrrd"
        nrrd.write(new_mask_path, np.int8(mask), header=head)
        count+=1
        # break
    break

In [ ]:
# The dataset.json contains metadata that nnU-Net needs for training. We have greatly reduced the number of required fields since version 1!
# Here is what the dataset.json should look like at the example of the Dataset005_Prostate from the MSD:
# { 
#  "channel_names": {  # formerly modalities
#    "0": "T2", 
#    "1": "ADC"
#  }, 
#  "labels": {  # THIS IS DIFFERENT NOW!
#    "background": 0,
#    "PZ": 1,
#    "TZ": 2
#  }, 
#  "numTraining": 32, 
#  "file_ending": ".nii.gz"
#  "overwrite_image_reader_writer": "SimpleITKIO"  # optional! If not provided nnU-Net will automatically determine the ReaderWriter
#  }

In [4]:
ending

'nrrd'

In [5]:
data = {}
data["channel_names"] = {"0":"CT"}
data["labels"] = {name:i+1 for i,name in enumerate(rois)}
data["labels"]["Background"] = 0
data["num_training"] = len(glob.glob(f"/Users/jmarsill/Documents/nnunetV2/{dataset_name}/imagesTr/*"))
data["file_ending"] = f".{ending}"

In [11]:
import json
with open(f"/Users/jmarsill/Documents/nnunetV2/{dataset_name}/dataset.json", 'w', encoding ='utf8') as json_file:
    json.dump(data, json_file)#, ensure_ascii = False)

In [23]:
import shutil
for i, v in enumerate(im):
    shutil.move(v.replace("imagesTr","labelsTr"), im_[i].replace("imagesTr","labelsTr"))